In [1]:
import dotenv

dotenv.load_dotenv()

True

In [8]:
import openai
import psycopg2
import json
import os

USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")

# PostgreSQL 연결 설정 - 이거 쓰는거 ㅇㅋㅋ
conn = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

# 커서 생성
cur = conn.cursor()

# 테이블에서 description 가져오기
cur.execute("SELECT id, description FROM documents;")
rows = cur.fetchall()

In [9]:
from openai import OpenAI
client = OpenAI()

for row in rows:
    doc_id, description = row

    response = client.embeddings.create(
         model="text-embedding-3-small",  # 512 차원 임베딩 모델
         input=description
     )

    embedding = response.data[0].embedding

    cur.execute("""
        UPDATE documents
       SET embedding = %s
       WHERE id = %s;
    """, (json.dumps(embedding), doc_id))

# 끝 !!
conn.commit()
cur.close()
conn.close()